<a href="https://colab.research.google.com/github/mhriyad99/smart-grid/blob/main/DataGeneration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import sys
import json
import warnings
import numpy as np
import pandas as pd
from dataclasses import dataclass
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.utils.validation import check_is_fitted, check_array
from typing import Dict, Any, List, Tuple, Optional
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import (MinMaxScaler, StandardScaler,
PowerTransformer, FunctionTransformer, KBinsDiscretizer)
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFE, SelectKBest, chi2, mutual_info_classif

from google.colab import drive
drive.mount('/content/drive')

RANDOM_STATE = 42
warnings.filterwarnings('ignore')

Mounted at /content/drive


# Data Generation

In [ ]:
class SquareTransformer:
    """Element-wise square transform."""
    def fit(self, X, y=None):
        X = np.asarray(X)
        self.n_features_in_ = X.shape[1]
        return self

    def transform(self, X):
        X = np.asarray(X)
        return np.square(X)

    def fit_transform(self, X, y=None):
        return self.fit(X, y).transform(X)

class PositiveShiftForBoxCox(BaseEstimator, TransformerMixin):
    """
    Ensure features are strictly positive for Box-Cox by shifting each column if needed.
    shift[i] = -(min_i) + epsilon if min_i <= 0 else 0
    """

    def __init__(self, epsilon=0.02):
        self.epsilon = epsilon

    def fit(self, X):
        # Validate input
        X = check_array(X, dtype=np.float64, force_all_finite=True)

        # Calculate minimum values for each feature
        mins = np.nanmin(X, axis=0)

        # Calculate shifts: if min <= 0, shift by -min + epsilon, otherwise 0
        self.shifts_ = np.where(mins <= 0.0, -mins + self.epsilon, 0.0)
        self.n_features_in_ = X.shape[1]

        # Store feature names if available
        if hasattr(X, 'columns'):
            self.feature_names_in_ = X.columns.tolist()

        return self

    def transform(self, X):
        # Check if fitted
        check_is_fitted(self, 'shifts_')

        # Validate input
        X = check_array(X, dtype=np.float64, force_all_finite=True)

        # Check feature consistency
        if X.shape[1] != self.n_features_in_:
            raise ValueError(
                f"X has {X.shape[1]} features, but PositiveShiftForBoxCox "
                f"is expecting {self.n_features_in_} features as seen in fit."
            )

        # Apply shifts
        X_shifted = X + self.shifts_

        # Verify all values are positive
        if np.any(X_shifted <= 0):
            problematic_features = np.where(np.any(X_shifted <= 0, axis=0))[0]
            raise ValueError(
                f"After shifting, some values are still <= 0 in features: {problematic_features}. "
                f"Consider increasing epsilon (current: {self.epsilon})."
            )

        return X_shifted

    def fit_transform(self, X):
        return self.fit(X).transform(X)

In [ ]:
data = pd.read_csv('/content/Data_for_UCI_named.csv')
data['stabf'] = data['stabf'].map({'stable': 1, 'unstable': 0})
X = data.drop(['stabf', 'stab'], axis=1)
y = data['stabf']
data.to_csv('raw_data.csv', index=False)

FileNotFoundError: [Errno 2] No such file or directory: '/content/Data_for_UCI_named.csv'

In [ ]:
squared_x = SquareTransformer().fit_transform(X)
squared_data = pd.DataFrame(squared_x, columns=X.columns)
squared_data.to_csv('squared_data.csv', index=False)

In [ ]:
box_cox_x = PositiveShiftForBoxCox().fit_transform(X)
box_cox_data = pd.DataFrame(box_cox_x, columns=X.columns)
box_cox_data.to_csv('box_cox_data.csv', index=False)

In [ ]:
std_x = StandardScaler().fit_transform(X)
std_data = pd.DataFrame(std_x, columns=X.columns)
std_data.to_csv('standard_data.csv', index=False)

In [ ]:
min_max_x = MinMaxScaler().fit_transform(X)
min_max_data = pd.DataFrame(min_max_x, columns=X.columns)
min_max_data.to_csv('min_max_data.csv', index=False)

# Selectors

In [ ]:
class CorrelationFilter(BaseEstimator, TransformerMixin):
    """
    Remove features with absolute Pearson correlation above a threshold (train-fold only).
    Keeps one feature from each correlated pair via a greedy strategy.
    """
    def __init__(self, threshold: float = 0.95):
        self.threshold = threshold

    def fit(self, X):
        self.column_names_ = X.columns
        X = pd.DataFrame(X, columns=self.column_names_)

        # Calculate correlation matrix
        corr = X.corr().abs()
        self.correlation_matrix_ = corr

        # Get upper triangle of correlation matrix
        upper = corr.where(np.triu(np.ones(corr.shape), k=1).astype(bool))

        # Find columns to drop and store correlation pairs
        to_drop = []
        self.high_correlations_ = []

        for column in upper.columns:
            high_corr_mask = upper[column] > self.threshold
            if high_corr_mask.any():
                # Store information about high correlations
                for idx in upper.index[high_corr_mask]:
                    self.high_correlations_.append({
                        'feature1': idx,
                        'feature2': column,
                        'correlation': corr.loc[idx, column],
                        'dropped': column
                    })
                to_drop.append(column)

        # Store indices to keep
        self.keep_indices_ = [i for i in range(X.shape[1]) if i not in set([X.columns.get_loc(c) for c in to_drop])]

        # Store column information
        self.dropped_columns_ = to_drop
        self.selected_columns_ = [self.column_names_[i] for i in self.keep_indices_]

        self.n_features_in_ = X.shape[1]
        self.n_features_out_ = len(self.keep_indices_)

        return self

    def transform(self, X):
        X = pd.DataFrame(X, columns=self.column_names_)
        return X.iloc[:, self.keep_indices_]

    def get_feature_selection_info(self):
        """
        Returns detailed information about feature selection process.

        Returns:
        --------
        dict: Dictionary containing selection metrics and information
        """
        info = {
            'n_features_original': self.n_features_in_,
            'n_features_selected': self.n_features_out_,
            'n_features_dropped': self.n_features_in_ - self.n_features_out_,
            'threshold_used': self.threshold,
            'selected_columns': self.selected_columns_,
            'dropped_columns': self.dropped_columns_,
            'high_correlations': self.high_correlations_,
            'reduction_percentage': ((self.n_features_in_ - self.n_features_out_) / self.n_features_in_) * 100
        }
        return info


class MutualInfoSelector(BaseEstimator, TransformerMixin):
    """Select top-k features by mutual information."""
    def __init__(self, k: int = 8, random_state: int = RANDOM_STATE):
        self.k = k
        self.random_state = random_state
        self._selector = SelectKBest(score_func=lambda X, y: mutual_info_classif(X, y, random_state=self.random_state), k=self.k)

    def fit(self, X, y):
        self._selector.set_params(k=min(self.k, np.asarray(X).shape[1]))
        self._selector.fit(X, y)
        return self

    def transform(self, X):
        return self._selector.transform(X)

In [ ]:
class SafeChi2Selector(BaseEstimator, TransformerMixin):
    def __init__(self, k=None, n_bins=5, auto_select_k=True, cv_folds=5,
                 max_k=None, feature_names=None, random_state=42):

        self.k = k
        self.n_bins = n_bins
        self.auto_select_k = auto_select_k
        self.cv_folds = cv_folds
        self.max_k = max_k
        self.feature_names = feature_names
        self.random_state = random_state

        # Initialize components
        self._discretizer = KBinsDiscretizer(
            n_bins=n_bins,
            encode='ordinal',
            strategy='uniform',
            random_state=random_state
        )
        self._selector = None

        # Store results
        self.selected_features_ = None
        self.selected_indices_ = None
        self.feature_scores_ = None
        self.k_scores_ = None  # CV scores for different k values
        self.best_k_ = None

    def _get_feature_names(self, X):
        """Get feature names from input or use provided names."""
        if self.feature_names is not None:
            return self.feature_names
        elif hasattr(X, 'columns'):
            return list(X.columns)
        else:
            return [f'feature_{i}' for i in range(X.shape[1])]

    def _select_best_k(self, X_discrete, y):
        """Automatically select best k using cross-validation."""
        n_features = X_discrete.shape[1]
        max_k = self.max_k if self.max_k is not None else min(20, n_features)
        k_range = range(1, min(max_k + 1, n_features + 1))

        scores = []
        estimator = DecisionTreeClassifier(random_state=RANDOM_STATE)

        for k in k_range:
            try:
                selector = SelectKBest(score_func=chi2, k=k)
                X_selected = selector.fit_transform(X_discrete, y)

                # Use cross-validation to evaluate performance
                cv_scores = cross_val_score(
                    estimator, X_selected, y,
                    cv=self.cv_folds,
                    scoring='accuracy'
                )
                scores.append(cv_scores.mean())
            except:
                scores.append(0)  # Handle any errors

        self.k_scores_ = dict(zip(k_range, scores))
        best_k = k_range[np.argmax(scores)]

        print(f"Best k selected: {best_k} (CV score: {max(scores):.4f})")
        return best_k

    def fit(self, X, y):

        # Convert to numpy array if needed
        if hasattr(X, 'values'):
            X_array = X.values
        else:
            X_array = np.array(X)

        # Store feature names
        self.feature_names_in_ = self._get_feature_names(X)

        # Discretize continuous features
        X_discrete = self._discretizer.fit_transform(X_array)

        # Determine k
        if self.auto_select_k or self.k is None:
            self.best_k_ = self._select_best_k(X_discrete, y)
        else:
            self.best_k_ = min(self.k, X_array.shape[1])

        # Fit the selector with the chosen k
        self._selector = SelectKBest(score_func=chi2, k=self.best_k_)
        self._selector.fit(X_discrete, y)

        # Store selected feature information
        self.selected_indices_ = self._selector.get_support(indices=True)
        self.selected_features_ = [self.feature_names_in_[i] for i in self.selected_indices_]
        self.feature_scores_ = dict(zip(
            self.feature_names_in_,
            self._selector.scores_
        ))

        return self

    def transform(self, X):

        if self._selector is None:
            raise ValueError("Selector has not been fitted yet.")

        # Convert to numpy array if needed
        if hasattr(X, 'values'):
            X_array = X.values
        else:
            X_array = np.array(X)

        # Discretize and select features
        X_discrete = self._discretizer.transform(X_array)
        X_selected = self._selector.transform(X_discrete)

        return X_selected

    def fit_transform(self, X, y):
        """Fit the selector and transform the data."""
        return self.fit(X, y).transform(X)

    def get_feature_names_out(self, input_features=None):
        """Get output feature names for transformation."""
        if self.selected_features_ is None:
            raise ValueError("Selector has not been fitted yet.")
        return self.selected_features_

    def get_support(self, indices=False):
        """Get a mask or integer index of the selected features."""
        if self._selector is None:
            raise ValueError("Selector has not been fitted yet.")
        return self._selector.get_support(indices=indices)

    def get_feature_scores_df(self):

        if self.feature_scores_ is None:
            raise ValueError("Selector has not been fitted yet.")

        df = pd.DataFrame([
            {
                'feature': feature,
                'score': score,
                'selected': feature in self.selected_features_
            }
            for feature, score in self.feature_scores_.items()
        ])

        return df.sort_values('score', ascending=False)

    def plot_k_scores(self):
        """Plot cross-validation scores for different k values."""
        if self.k_scores_ is None:
            raise ValueError("No k scores available. Set auto_select_k=True or fit with k=None.")

        try:
            import matplotlib.pyplot as plt

            k_values = list(self.k_scores_.keys())
            scores = list(self.k_scores_.values())

            plt.figure(figsize=(10, 6))
            plt.plot(k_values, scores, 'o-')
            plt.axvline(x=self.best_k_, color='r', linestyle='--',
                       label=f'Best k = {self.best_k_}')
            plt.xlabel('Number of Features (k)')
            plt.ylabel('Cross-Validation Accuracy')
            plt.title('Feature Selection Performance vs Number of Features')
            plt.legend()
            plt.grid(True, alpha=0.3)
            plt.show()

        except ImportError:
            print("Matplotlib not available. Cannot plot k scores.")
            print("K scores:", self.k_scores_)

    def summary(self):
        """Print a summary of the feature selection results."""
        if self.selected_features_ is None:
            raise ValueError("Selector has not been fitted yet.")

        print(f"SafeChi2Selector Summary")
        print(f"=" * 40)
        print(f"Number of input features: {len(self.feature_names_in_)}")
        print(f"Number of selected features: {len(self.selected_features_)}")
        print(f"Best k: {self.best_k_}")

        if self.k_scores_:
            print(f"Best CV score: {max(self.k_scores_.values()):.4f}")

        print(f"\nSelected features:")
        for i, feature in enumerate(self.selected_features_, 1):
            score = self.feature_scores_[feature]
            print(f"  {i:2d}. {feature:<20} (score: {score:.4f})")

In [ ]:
class MutualInfoSelector(BaseEstimator, TransformerMixin):
    """Enhanced Mutual Information feature selector with automatic K selection."""

    def __init__(self, k=None, auto_select_k=True, cv_folds=5, max_k=None,
                 feature_names=None, random_state=42, discrete_features='auto'):

        self.k = k
        self.auto_select_k = auto_select_k
        self.cv_folds = cv_folds
        self.max_k = max_k
        self.feature_names = feature_names
        self.random_state = random_state
        self.discrete_features = discrete_features

        # Initialize components
        self._selector = None

        # Store results
        self.selected_features_ = None
        self.selected_indices_ = None
        self.feature_scores_ = None
        self.k_scores_ = None  # CV scores for different k values
        self.best_k_ = None

    def _get_feature_names(self, X):
        """Get feature names from input or use provided names."""
        if self.feature_names is not None:
            return self.feature_names
        elif hasattr(X, 'columns'):
            return list(X.columns)
        else:
            return [f'feature_{i}' for i in range(X.shape[1])]

    def _mutual_info_score_func(self, X, y):
        """Wrapper for mutual_info_classif with consistent random state."""
        return mutual_info_classif(
            X, y,
            discrete_features=self.discrete_features,
            random_state=self.random_state
        )

    def _select_best_k(self, X, y):
        """Automatically select best k using cross-validation."""
        n_features = X.shape[1]
        max_k = self.max_k if self.max_k is not None else min(20, n_features)
        k_range = range(1, min(max_k + 1, n_features + 1))

        scores = []
        estimator = DecisionTreeClassifier(random_state=RANDOM_STATE)

        print(f"Evaluating k from 1 to {min(max_k, n_features)}...")

        for i, k in enumerate(k_range):
            try:
                selector = SelectKBest(score_func=self._mutual_info_score_func, k=k)
                X_selected = selector.fit_transform(X, y)

                # Use cross-validation to evaluate performance
                cv_scores = cross_val_score(
                    estimator, X_selected, y,
                    cv=self.cv_folds,
                    scoring='accuracy'
                )
                scores.append(cv_scores.mean())

                # Progress indicator
                if (i + 1) % 5 == 0 or i == len(k_range) - 1:
                    print(f"  Evaluated {i + 1}/{len(k_range)} values...")

            except Exception as e:
                print(f"  Warning: Error evaluating k={k}: {str(e)}")
                scores.append(0)  # Handle any errors

        self.k_scores_ = dict(zip(k_range, scores))
        best_k = k_range[np.argmax(scores)]

        print(f"Best k selected: {best_k} (CV score: {max(scores):.4f})")
        return best_k

    def fit(self, X, y):

        # Convert to numpy array if needed
        if hasattr(X, 'values'):
            X_array = X.values
        else:
            X_array = np.array(X)

        # Store feature names
        self.feature_names_in_ = self._get_feature_names(X)

        # Determine k
        if self.auto_select_k or self.k is None:
            self.best_k_ = self._select_best_k(X_array, y)
        else:
            self.best_k_ = min(self.k, X_array.shape[1])

        # Fit the selector with the chosen k
        self._selector = SelectKBest(score_func=self._mutual_info_score_func, k=self.best_k_)
        self._selector.fit(X_array, y)

        # Store selected feature information
        self.selected_indices_ = self._selector.get_support(indices=True)
        self.selected_features_ = [self.feature_names_in_[i] for i in self.selected_indices_]
        self.feature_scores_ = dict(zip(
            self.feature_names_in_,
            self._selector.scores_
        ))

        return self

    def transform(self, X):

        if self._selector is None:
            raise ValueError("Selector has not been fitted yet.")

        # Convert to numpy array if needed
        if hasattr(X, 'values'):
            X_array = X.values
        else:
            X_array = np.array(X)

        return self._selector.transform(X_array)

    def fit_transform(self, X, y):
        """Fit the selector and transform the data."""
        return self.fit(X, y).transform(X)

    def get_feature_names_out(self, input_features=None):
        """Get output feature names for transformation."""
        if self.selected_features_ is None:
            raise ValueError("Selector has not been fitted yet.")
        return self.selected_features_

    def get_support(self, indices=False):
        """Get a mask or integer index of the selected features."""
        if self._selector is None:
            raise ValueError("Selector has not been fitted yet.")
        return self._selector.get_support(indices=indices)

    def get_feature_scores_df(self):

        if self.feature_scores_ is None:
            raise ValueError("Selector has not been fitted yet.")

        df = pd.DataFrame([
            {
                'feature': feature,
                'mutual_info_score': score,
                'selected': feature in self.selected_features_
            }
            for feature, score in self.feature_scores_.items()
        ])

        return df.sort_values('mutual_info_score', ascending=False)

    def plot_k_scores(self):
        """Plot cross-validation scores for different k values."""
        if self.k_scores_ is None:
            raise ValueError("No k scores available. Set auto_select_k=True or fit with k=None.")

        try:
            import matplotlib.pyplot as plt

            k_values = list(self.k_scores_.keys())
            scores = list(self.k_scores_.values())

            plt.figure(figsize=(10, 6))
            plt.plot(k_values, scores, 'o-', color='green')
            plt.axvline(x=self.best_k_, color='r', linestyle='--',
                       label=f'Best k = {self.best_k_}')
            plt.xlabel('Number of Features (k)')
            plt.ylabel('Cross-Validation Accuracy')
            plt.title('Mutual Information Feature Selection Performance vs Number of Features')
            plt.legend()
            plt.grid(True, alpha=0.3)
            plt.show()

        except ImportError:
            print("Matplotlib not available. Cannot plot k scores.")
            print("K scores:", self.k_scores_)

    def plot_feature_scores(self, top_n=15):
        """Plot the mutual information scores for the top N features."""
        if self.feature_scores_ is None:
            raise ValueError("Selector has not been fitted yet.")

        try:
            import matplotlib.pyplot as plt

            # Get top features
            df = self.get_feature_scores_df()
            top_features = df.head(top_n)

            # Create plot
            plt.figure(figsize=(10, 6))
            colors = ['red' if selected else 'skyblue' for selected in top_features['selected']]
            bars = plt.bar(range(len(top_features)), top_features['mutual_info_score'], color=colors)

            # Customize plot
            plt.xlabel('Features')
            plt.ylabel('Mutual Information Score')
            plt.title(f'Top {top_n} Features by Mutual Information Score')
            plt.xticks(range(len(top_features)), top_features['feature'], rotation=45, ha='right')

            # Add legend
            red_patch = plt.Rectangle((0, 0), 1, 1, fc='red', label='Selected')
            blue_patch = plt.Rectangle((0, 0), 1, 1, fc='skyblue', label='Not Selected')
            plt.legend(handles=[red_patch, blue_patch])

            plt.tight_layout()
            plt.show()

        except ImportError:
            print("Matplotlib not available. Cannot plot feature scores.")
            print("Top features:")
            print(self.get_feature_scores_df().head(top_n))

    def summary(self):
        """Print a summary of the feature selection results."""
        if self.selected_features_ is None:
            raise ValueError("Selector has not been fitted yet.")

        print(f"MutualInfoSelector Summary")
        print(f"=" * 40)
        print(f"Number of input features: {len(self.feature_names_in_)}")
        print(f"Number of selected features: {len(self.selected_features_)}")
        print(f"Best k: {self.best_k_}")

        if self.k_scores_:
            print(f"Best CV score: {max(self.k_scores_.values()):.4f}")

        print(f"\nSelected features (by mutual information score):")
        selected_scores = [(feature, self.feature_scores_[feature])
                          for feature in self.selected_features_]
        selected_scores.sort(key=lambda x: x[1], reverse=True)

        for i, (feature, score) in enumerate(selected_scores, 1):
            print(f"  {i:2d}. {feature:<20} (MI score: {score:.6f})")

        # Show some stats
        all_scores = list(self.feature_scores_.values())
        print(f"\nMutual Information Score Statistics:")
        print(f"  Mean: {np.mean(all_scores):.6f}")
        print(f"  Std:  {np.std(all_scores):.6f}")
        print(f"  Min:  {np.min(all_scores):.6f}")
        print(f"  Max:  {np.max(all_scores):.6f}")

# Selection

In [ ]:
raw_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Smart Grid/Data/raw_data.csv')
squared_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Smart Grid/Data/squared_data.csv')
min_max_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Smart Grid/Data/min_max_data.csv')
box_cox_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Smart Grid/Data/box_cox_data.csv')
standard_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Smart Grid/Data/standard_data.csv')

In [ ]:
corr_fs = CorrelationFilter()
corr_fs.fit(standard_data)
corr_selected_features = corr_fs.get_feature_selection_info()
corr_selected_features

{'n_features_original': 12,
 'n_features_selected': 12,
 'n_features_dropped': 0,
 'threshold_used': 0.95,
 'selected_columns': ['tau1',
  'tau2',
  'tau3',
  'tau4',
  'p1',
  'p2',
  'p3',
  'p4',
  'g1',
  'g2',
  'g3',
  'g4'],
 'dropped_columns': [],
 'high_correlations': [],
 'reduction_percentage': 0.0}

In [ ]:
data = raw_data.copy()
X = data.drop(['stab', 'stabf'], axis=1)
y = data['stabf']
# data.to_csv('raw_data.csv', index=False)

## Chi2

In [ ]:
raw_chi2_selector = SafeChi2Selector(
        auto_select_k=True,
        feature_names=X.columns,
        n_bins=5
    )

raw_chi2_selected = raw_chi2_selector.fit_transform(min_max_data, y)
raw_chi2_selector.summary()

Best k selected: 8 (CV score: 0.8514)
SafeChi2Selector Summary
Number of input features: 12
Number of selected features: 8
Best k: 8
Best CV score: 0.8514

Selected features:
   1. tau1                 (score: 522.8087)
   2. tau2                 (score: 586.5438)
   3. tau3                 (score: 544.5617)
   4. tau4                 (score: 547.0902)
   5. g1                   (score: 379.1197)
   6. g2                   (score: 462.7384)
   7. g3                   (score: 515.8856)
   8. g4                   (score: 388.3426)


In [ ]:
box_cox_chi2_selector = SafeChi2Selector(
        auto_select_k=True,
        feature_names=X.columns,
        n_bins=5
    )

box_cox_selected = box_cox_chi2_selector.fit_transform(box_cox_data, y)
box_cox_chi2_selector.summary()

Best k selected: 8 (CV score: 0.8514)
SafeChi2Selector Summary
Number of input features: 12
Number of selected features: 8
Best k: 8
Best CV score: 0.8514

Selected features:
   1. tau1                 (score: 522.8087)
   2. tau2                 (score: 586.5438)
   3. tau3                 (score: 544.5617)
   4. tau4                 (score: 547.0902)
   5. g1                   (score: 379.1197)
   6. g2                   (score: 462.7384)
   7. g3                   (score: 515.8856)
   8. g4                   (score: 388.3426)


In [ ]:
min_max_chi2_selector = SafeChi2Selector(
        auto_select_k=True,
        feature_names=X.columns,
        n_bins=5
    )

min_max_selected = min_max_chi2_selector.fit_transform(min_max_data, y)
min_max_chi2_selector.summary()

Best k selected: 8 (CV score: 0.8514)
SafeChi2Selector Summary
Number of input features: 12
Number of selected features: 8
Best k: 8
Best CV score: 0.8514

Selected features:
   1. tau1                 (score: 522.8087)
   2. tau2                 (score: 586.5438)
   3. tau3                 (score: 544.5617)
   4. tau4                 (score: 547.0902)
   5. g1                   (score: 379.1197)
   6. g2                   (score: 462.7384)
   7. g3                   (score: 515.8856)
   8. g4                   (score: 388.3426)


In [ ]:
standard_chi2_selector = SafeChi2Selector(
        auto_select_k=True,
        feature_names=X.columns,
        n_bins=5
    )

standard_chi2_selected = standard_chi2_selector.fit_transform(standard_data, y)
standard_chi2_selector.summary()

Best k selected: 8 (CV score: 0.8514)
SafeChi2Selector Summary
Number of input features: 12
Number of selected features: 8
Best k: 8
Best CV score: 0.8514

Selected features:
   1. tau1                 (score: 522.8087)
   2. tau2                 (score: 586.5438)
   3. tau3                 (score: 544.5617)
   4. tau4                 (score: 547.0902)
   5. g1                   (score: 379.1197)
   6. g2                   (score: 462.7384)
   7. g3                   (score: 515.8856)
   8. g4                   (score: 388.3426)


In [ ]:
squared_chi2_selector = SafeChi2Selector(
        auto_select_k=True,
        feature_names=X.columns,
        n_bins=5
    )

squared_chi2_selected = squared_chi2_selector.fit_transform(squared_data, y)
squared_chi2_selector.summary()

Best k selected: 8 (CV score: 0.7731)
SafeChi2Selector Summary
Number of input features: 12
Number of selected features: 8
Best k: 8
Best CV score: 0.7731

Selected features:
   1. tau1                 (score: 411.5204)
   2. tau2                 (score: 515.9214)
   3. tau3                 (score: 484.7504)
   4. tau4                 (score: 447.1249)
   5. g1                   (score: 510.6784)
   6. g2                   (score: 609.1546)
   7. g3                   (score: 722.4594)
   8. g4                   (score: 512.0628)


In [ ]:
chi2_metrics_list = [raw_chi2_selector.feature_scores_, box_cox_chi2_selector.feature_scores_,
              min_max_chi2_selector.feature_scores_, standard_chi2_selector.feature_scores_,
              squared_chi2_selector.feature_scores_]
chi2_metrics_df = pd.DataFrame(chi2_metrics_list, index=['raw', 'box_cox', 'min_max', 'standard', 'squared'])
chi2_metrics = chi2_metrics_df.T


In [ ]:
chi2_metrics

,raw,box_cox,min_max,standard,squared
tau1,522.808674,522.808674,522.808674,522.808674,411.520423
tau2,586.543757,586.543757,586.543757,586.543757,515.921390
tau3,544.561735,544.561735,544.561735,544.561735,484.750411
tau4,547.090209,547.090209,547.090209,547.090209,447.124877
p1,0.517785,0.517785,0.517785,0.517785,0.775844
p2,0.832193,0.832193,0.832193,0.832193,0.106387
p3,0.026195,0.026195,0.026195,0.026195,0.058577
p4,5.893980,5.893980,5.893980,5.893980,5.285761
g1,379.119707,379.119707,379.119707,379.119707,510.678430
g2,462.738357,462.738357,462.738357,462.738357,609.154646


In [ ]:
chi2_metrics.to_csv('/content/drive/MyDrive/Colab Notebooks/Smart Grid/Data/chi2_fs_metrics.csv')

## Mutual Information Gain

In [ ]:
raw_mu_selector = MutualInfoSelector(
        auto_select_k=True,
        feature_names=X.columns,
        max_k=12,
        cv_folds=5,
        random_state=RANDOM_STATE
    )

raw_mu_selected = raw_mu_selector.fit_transform(X, y)
raw_mu_selector.summary()

Evaluating k from 1 to 12...
  Evaluated 5/12 values...
  Evaluated 10/12 values...
  Evaluated 12/12 values...
Best k selected: 8 (CV score: 0.8506)
MutualInfoSelector Summary
Number of input features: 12
Number of selected features: 8
Best k: 8
Best CV score: 0.8506

Selected features (by mutual information score):
   1. tau4                 (MI score: 0.043882)
   2. tau1                 (MI score: 0.040263)
   3. tau2                 (MI score: 0.035675)
   4. tau3                 (MI score: 0.034338)
   5. g2                   (MI score: 0.028857)
   6. g3                   (MI score: 0.027905)
   7. g1                   (MI score: 0.020657)
   8. g4                   (MI score: 0.017152)

Mutual Information Score Statistics:
  Mean: 0.020911
  Std:  0.016037
  Min:  0.000000
  Max:  0.043882


In [ ]:
box_cox_mu_selector = MutualInfoSelector(
        auto_select_k=True,
        feature_names=X.columns,
        max_k=12,
        cv_folds=5,
        random_state=RANDOM_STATE
    )

box_cox_mu_selected = box_cox_mu_selector.fit_transform(box_cox_data, y)
box_cox_mu_selector.summary()

Evaluating k from 1 to 12...
  Evaluated 5/12 values...
  Evaluated 10/12 values...
  Evaluated 12/12 values...
Best k selected: 8 (CV score: 0.8506)
MutualInfoSelector Summary
Number of input features: 12
Number of selected features: 8
Best k: 8
Best CV score: 0.8506

Selected features (by mutual information score):
   1. tau4                 (MI score: 0.043882)
   2. tau1                 (MI score: 0.040263)
   3. tau2                 (MI score: 0.035675)
   4. tau3                 (MI score: 0.034338)
   5. g2                   (MI score: 0.028857)
   6. g3                   (MI score: 0.027905)
   7. g1                   (MI score: 0.020657)
   8. g4                   (MI score: 0.017152)

Mutual Information Score Statistics:
  Mean: 0.020911
  Std:  0.016037
  Min:  0.000000
  Max:  0.043882


In [ ]:
min_max_mu_selector = MutualInfoSelector(
        auto_select_k=True,
        feature_names=X.columns,
        max_k=12,
        cv_folds=5,
        random_state=RANDOM_STATE
    )

min_max_mu_selected = min_max_mu_selector.fit_transform(min_max_data, y)
min_max_mu_selector.summary()

Evaluating k from 1 to 12...
  Evaluated 5/12 values...
  Evaluated 10/12 values...
  Evaluated 12/12 values...
Best k selected: 8 (CV score: 0.8506)
MutualInfoSelector Summary
Number of input features: 12
Number of selected features: 8
Best k: 8
Best CV score: 0.8506

Selected features (by mutual information score):
   1. tau4                 (MI score: 0.043882)
   2. tau1                 (MI score: 0.040263)
   3. tau2                 (MI score: 0.035675)
   4. tau3                 (MI score: 0.034338)
   5. g2                   (MI score: 0.028857)
   6. g3                   (MI score: 0.027905)
   7. g1                   (MI score: 0.020657)
   8. g4                   (MI score: 0.017152)

Mutual Information Score Statistics:
  Mean: 0.020911
  Std:  0.016037
  Min:  0.000000
  Max:  0.043882


In [ ]:
standard_mu_selector = MutualInfoSelector(
        auto_select_k=True,
        feature_names=X.columns,
        max_k=12,
        cv_folds=5,
        random_state=RANDOM_STATE
    )

standard_mu_selected = standard_mu_selector.fit_transform(standard_data, y)
standard_mu_selector.summary()

Evaluating k from 1 to 12...
  Evaluated 5/12 values...
  Evaluated 10/12 values...
  Evaluated 12/12 values...
Best k selected: 8 (CV score: 0.8506)
MutualInfoSelector Summary
Number of input features: 12
Number of selected features: 8
Best k: 8
Best CV score: 0.8506

Selected features (by mutual information score):
   1. tau4                 (MI score: 0.043882)
   2. tau1                 (MI score: 0.040263)
   3. tau2                 (MI score: 0.035675)
   4. tau3                 (MI score: 0.034338)
   5. g2                   (MI score: 0.028857)
   6. g3                   (MI score: 0.027905)
   7. g1                   (MI score: 0.020657)
   8. g4                   (MI score: 0.017152)

Mutual Information Score Statistics:
  Mean: 0.020911
  Std:  0.016037
  Min:  0.000000
  Max:  0.043882


In [ ]:
squared_mu_selector = MutualInfoSelector(
        auto_select_k=True,
        feature_names=X.columns,
        max_k=12,
        cv_folds=5,
        random_state=RANDOM_STATE
    )

squared_mu_selected = squared_mu_selector.fit_transform(squared_data, y)
squared_mu_selector.summary()

Evaluating k from 1 to 12...
  Evaluated 5/12 values...
  Evaluated 10/12 values...
  Evaluated 12/12 values...
Best k selected: 8 (CV score: 0.8503)
MutualInfoSelector Summary
Number of input features: 12
Number of selected features: 8
Best k: 8
Best CV score: 0.8503

Selected features (by mutual information score):
   1. tau4                 (MI score: 0.043825)
   2. tau1                 (MI score: 0.040194)
   3. tau2                 (MI score: 0.035599)
   4. tau3                 (MI score: 0.034230)
   5. g2                   (MI score: 0.028839)
   6. g3                   (MI score: 0.027902)
   7. g1                   (MI score: 0.020727)
   8. g4                   (MI score: 0.017026)

Mutual Information Score Statistics:
  Mean: 0.020875
  Std:  0.016016
  Min:  0.000000
  Max:  0.043825


In [ ]:
mu_metrics_list = [raw_mu_selector.feature_scores_, box_cox_mu_selector.feature_scores_,
              min_max_mu_selector.feature_scores_, standard_mu_selector.feature_scores_,
              squared_mu_selector.feature_scores_]
mu_metrics_df = pd.DataFrame(mu_metrics_list, index=['raw', 'box_cox', 'min_max', 'standard', 'squared'])
mu_metrics = mu_metrics_df.T
mu_metrics

,raw,box_cox,min_max,standard,squared
tau1,0.040263,0.040263,0.040263,0.040263,0.040194
tau2,0.035675,0.035675,0.035675,0.035675,0.035599
tau3,0.034338,0.034338,0.034338,0.034338,0.034230
tau4,0.043882,0.043882,0.043882,0.043882,0.043825
p1,0.000884,0.000884,0.000884,0.000884,0.000885
p2,0.000000,0.000000,0.000000,0.000000,0.000000
p3,0.001317,0.001317,0.001317,0.001317,0.001274
p4,0.000000,0.000000,0.000000,0.000000,0.000000
g1,0.020657,0.020657,0.020657,0.020657,0.020727
g2,0.028857,0.028857,0.028857,0.028857,0.028839


In [ ]:
mu_metrics.to_csv('/content/drive/MyDrive/Colab Notebooks/Smart Grid/Data/mu_fs_metrics.csv')

## RFE

In [ ]:
raw_rfe_selector = RFE(estimator=DecisionTreeClassifier(random_state=RANDOM_STATE),
                  n_features_to_select=8, step=1, verbose=0)
raw_rfe_selector.fit(X, y)
raw_rfe_support = raw_rfe_selector.get_support()
raw_rfe_feature = X.loc[:,raw_rfe_support].columns.tolist()
print(f'selected features: {raw_rfe_feature}')

selected features: ['tau1', 'tau2', 'tau3', 'tau4', 'g1', 'g2', 'g3', 'g4']


In [ ]:
raw_rfe_selector.ranking_

array([1, 1, 1, 1, 2, 5, 3, 4, 1, 1, 1, 1])

In [ ]:
box_cox_rfe_selector = RFE(estimator=DecisionTreeClassifier(random_state=RANDOM_STATE),
                  n_features_to_select=8, step=1, verbose=0)
box_cox_rfe_selector.fit(box_cox_data, y)
box_cox_rfe_support = box_cox_rfe_selector.get_support()
box_cox_rfe_feature = box_cox_data.loc[:,box_cox_rfe_support].columns.tolist()
print(f'selected features: {box_cox_rfe_feature}')

selected features: ['tau1', 'tau2', 'tau3', 'tau4', 'g1', 'g2', 'g3', 'g4']


In [ ]:
min_max_rfe_selector = RFE(estimator=DecisionTreeClassifier(random_state=RANDOM_STATE),
                  n_features_to_select=8, step=1, verbose=0)
min_max_rfe_selector.fit(min_max_data, y)
min_max_rfe_support = min_max_rfe_selector.get_support()
min_max_rfe_feature = min_max_data.loc[:,min_max_rfe_support].columns.tolist()
print(f'selected features: {min_max_rfe_feature}')

selected features: ['tau1', 'tau2', 'tau3', 'tau4', 'g1', 'g2', 'g3', 'g4']


In [ ]:
standard_rfe_selector = RFE(estimator=DecisionTreeClassifier(random_state=RANDOM_STATE),
                  n_features_to_select=8, step=1, verbose=0)
standard_rfe_selector.fit(standard_data, y)
standard_rfe_support = standard_rfe_selector.get_support()
standard_rfe_feature = standard_data.loc[:,standard_rfe_support].columns.tolist()
print(f'selected features: {standard_rfe_feature}')

selected features: ['tau1', 'tau2', 'tau3', 'tau4', 'g1', 'g2', 'g3', 'g4']


In [ ]:
squared_rfe_selector = RFE(estimator=DecisionTreeClassifier(random_state=RANDOM_STATE),
                  n_features_to_select=8, step=1, verbose=0)
squared_rfe_selector.fit(squared_data, y)
squared_rfe_support = squared_rfe_selector.get_support()
squared_rfe_feature = squared_data.loc[:,squared_rfe_support].columns.tolist()
print(f'selected features: {squared_rfe_feature}')

selected features: ['tau1', 'tau2', 'tau3', 'tau4', 'g1', 'g2', 'g3', 'g4']


In [ ]:
pd.DataFrame({
    'Feature': X.columns,
    'raw': raw_rfe_selector.ranking_,
    'box_cox': box_cox_rfe_selector.ranking_,
    'min_max': min_max_rfe_selector.ranking_,
    'standard': standard_rfe_selector.ranking_,
    'squared': squared_rfe_selector.ranking_
})

,Feature,raw,box_cox,min_max,standard,squared
0,tau1,1,1,1,1,1
1,tau2,1,1,1,1,1
2,tau3,1,1,1,1,1
3,tau4,1,1,1,1,1
4,p1,2,2,2,2,2
5,p2,5,5,5,5,5
6,p3,3,3,3,3,3
7,p4,4,4,4,4,4
8,g1,1,1,1,1,1
9,g2,1,1,1,1,1


In [ ]:
from sklearn.neural_network import MLPClassifier

In [ ]:
mlp = MLPClassifier(random_state=42, activation = 'relu', alpha= 0.0001, hidden_layer_sizes= (100,), learning_rate_init= 0.001)
mlp.fit(X,y)

MLPClassifier(random_state=42)

In [ ]:
mlp.feature_importance_